In [1]:
"DOING THE SAME AS FOR THE MME_DATASET, BUT THEN FOR ONLY COUNTRIES *NOT* IN THE WESTERN LIST"

'DOING THE SAME AS FOR THE MME_DATASET, BUT THEN FOR ONLY COUNTRIES *NOT* IN THE WESTERN LIST'

In [2]:
import pandas as pd

# create a dataframe with only the UserCountry3 column from the 71M SharedResponses.csv file
reader_country = pd.read_csv('SharedResponses.csv', usecols=['UserCountry3', 'ResponseID'])

In [3]:
# List of countries considered 'Western' from the UN

countries = ["AND",  "AUS", "AUT", "BEL", "CAN", "CHE", "DNK", "DEU", "ESP", "FIN",  "FRA",  "GBR",  "GRC",  "IRL", 
            "ISR", "ISL", "ITA", "LIE", "LUX", "MLT", "MCO", "NLD", "NOR", "NZL", "PRT", "SMR", "SWE", "TUR", "USA" ]

# Andorra, Australia, Austria, Belgium, Canada, Switzerland, Denmark, Germany, Spain, Finland, France, United Kingdom, Greece, Ireland, 
# Israel, Iceland, Italy, Liechtenstein, Luxembourg, Malta, Monaco, Netherlands, Norway, New Zealand, Portugal, San Marino, Sweden, Turkey, USA

In [4]:
# Filter the rows where UserCountryID is NOT the list of countries that are considered Western

non_western = reader_country[~reader_country['UserCountry3'].isin(countries)]

In [5]:
non_western.shape

(21837017, 2)

In [6]:
# only keep complete scenarios of 2

# Count how many times each ResponseID appears
response_counts = non_western['ResponseID'].value_counts()

# Get the ResponseIDs that appear exactly twice
ids_to_keep = response_counts[response_counts == 2].index

# Filter the rows where ResponseID is in the list of ids_to_keep
complete_responseid_country = non_western[non_western['ResponseID'].isin(ids_to_keep)]

In [7]:
print(complete_responseid_country.value_counts().min())
print(complete_responseid_country.value_counts().max())

2
2


In [8]:
complete_responseid_country.shape

(21084418, 2)

In [9]:
# checking the NAN's in UserCountry3 in the complete_responseid_country dataset and removing them
print(complete_responseid_country['UserCountry3'].isna().sum())

644868


In [12]:
# removing the NAN's
complete_responseid_country = complete_responseid_country.dropna(subset=['UserCountry3'])
print(complete_responseid_country['UserCountry3'].isna().sum())
print(complete_responseid_country.shape)

0
(20439550, 2)


In [63]:
# Drop duplicates based on 'ResponseID' and keep the first occurrence
complete_responseid_country = complete_responseid_country.drop_duplicates(subset=['ResponseID'], keep='first')

In [14]:
country_subset = complete_responseid_country.sample(n=3500000)  # want 7M rows (with accounting for NAn's and deleting rows with 'random' (around 10%)), so 17M / 2 = 8.5M ResponseID's necessary

In [15]:
# checking if they are all unique

country_subset.nunique()

ResponseID      3500000
UserCountry3        202
dtype: int64

In [16]:
# transform to a list to feed to the for loop that will extract all the corresponding columns from the SharedResponses.csv file

country_sub_list = country_subset['ResponseID'].tolist()
# check if it went okay
print(len(country_sub_list))     # should be 8500000

3500000


In [17]:
subset_country = pd.DataFrame()

In [18]:
# reading SharedResponses to extract the rows with ResponseID's in country_sub_list
# this will result in a dataframe 'subset' that contains around 17 million rows (8,5 * 2)

chunk_size = 500_000
reader = pd.read_csv('SharedResponses.csv', chunksize=chunk_size, dtype=str, low_memory=False)

for i, chunk in enumerate(reader):
    
    print(f"Processing chunk {i+1}")

    # Filter rows where ResponseID is in reader_subset
    subset_chunk = chunk[chunk['ResponseID'].isin(country_sub_list)]

    # Append filtered chunk to empty df
    subset_country = pd.concat([subset_country, subset_chunk], ignore_index=True)

    print(f"Finished processing chunk {i+1}")

print("All chunks have been processed and combined.")

Processing chunk 1
Finished processing chunk 1
Processing chunk 2
Finished processing chunk 2
Processing chunk 3
Finished processing chunk 3
Processing chunk 4
Finished processing chunk 4
Processing chunk 5
Finished processing chunk 5
Processing chunk 6
Finished processing chunk 6
Processing chunk 7
Finished processing chunk 7
Processing chunk 8
Finished processing chunk 8
Processing chunk 9
Finished processing chunk 9
Processing chunk 10
Finished processing chunk 10
Processing chunk 11
Finished processing chunk 11
Processing chunk 12
Finished processing chunk 12
Processing chunk 13
Finished processing chunk 13
Processing chunk 14
Finished processing chunk 14
Processing chunk 15
Finished processing chunk 15
Processing chunk 16
Finished processing chunk 16
Processing chunk 17
Finished processing chunk 17
Processing chunk 18
Finished processing chunk 18
Processing chunk 19
Finished processing chunk 19
Processing chunk 20
Finished processing chunk 20
Processing chunk 21
Finished processin

In [19]:
subset_country.shape  # 7M rows, 41 columns

(7000000, 41)

In [20]:
# checking to see if the UserCountry3 column only contains countries not in the Western list
print(subset_country['UserCountry3'].isin(countries).value_counts())

# should be False for all rows (17M)

UserCountry3
False    7000000
Name: count, dtype: int64


In [21]:
subset_country_csv = subset_country.to_csv('subset_country.csv', index=False)

In [22]:
import pandas as pd
df_country = pd.read_csv('subset_country.csv')

C:\Users\esmku\AppData\Local\Temp\ipykernel_28492\2704394548.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_country = pd.read_csv('subset_country.csv')


In [23]:
df_country['ScenarioTypeStrict'].value_counts()

ScenarioTypeStrict
Utilitarian      1257090
Gender           1235454
Age              1235056
Species          1234144
Fitness          1232254
Random            615572
Social Status     190430
Name: count, dtype: int64

In [24]:
# removing all rows with 'random' in the ScenarioTypeStrict column

df_country = df_country[df_country['ScenarioTypeStrict'] != 'Random']

In [28]:
df_country['ScenarioTypeStrict'].value_counts()

ScenarioTypeStrict
Utilitarian      1257090
Gender           1235454
Age              1235056
Species          1234144
Fitness          1232254
Social Status     190430
Name: count, dtype: int64

In [25]:
# removing all rows with 'random' in the ScenarioTypeStrict column, as LLM's don't have a random scenario
df_country['AttributeLevel'].value_counts()

AttributeLevel
More       628545
Less       628545
Male       617727
Female     617727
Pets       617071
Hoomans    617071
Old        596848
Young      596848
Fat        556024
Fit        556024
Rand       203272
Low         74363
High        74363
Name: count, dtype: int64

In [32]:
df_country = df_country[df_country['AttributeLevel'] != 'Rand']

In [33]:
# removing all rows with 'Rand' in the AttributeLevel column, as LLM's don't have a random scenario
df_country['AttributeLevel'].value_counts()

AttributeLevel
More       628545
Less       628545
Male       617727
Female     617727
Pets       617071
Hoomans    617071
Old        596848
Young      596848
Fat        556024
Fit        556024
Low         74363
High        74363
Name: count, dtype: int64

In [34]:
df_country.shape

(6181156, 41)

In [35]:
df_country.isna().sum()

ResponseID                      0
ExtendedSessionID               0
UserID                        350
ScenarioOrder                   0
Intervention                    0
PedPed                          0
Barrier                         0
CrossingSignal                  0
AttributeLevel                  0
ScenarioTypeStrict              0
ScenarioType                    0
DefaultChoice                   0
NonDefaultChoice                0
DefaultChoiceIsOmission         0
NumberOfCharacters              0
DiffNumberOFCharacters          0
Saved                           0
Template                   740348
DescriptionShown           740348
LeftHand                   740348
UserCountry3                    0
Man                             0
Woman                           0
Pregnant                        0
Stroller                        0
OldMan                          0
OldWoman                        0
Boy                             0
Girl                            0
Homeless      

In [36]:
# deleting rows with NaN's in the UserID column

df_country = df_country.dropna(subset=['UserID'])

In [37]:
df_country.shape # should be 6181156 - 350 = 6180806

(6180806, 41)

In [39]:
# the total dataset has to be 5M rows.
# 2% of that is LLM's, so 100.000 rows
# the other 98% will be humans, so 4.900.000 rows

# need to subset 4900000 rows from the 6180806 rows
# need to delete 6180806 - 4900000 = 1280806 rows

In [40]:
# randomly delete 1280806 / 2 = 640.403 unique UserIDs (is 1280806 rows), to ensure 2% of the dataset is LLMs and 98% humans

# Getting unique UserIDs
Response_unique = df_country['ResponseID'].unique()
print(len(Response_unique))  # should be 6180806/2 = 3090403

# Selecting 559467 UserIDs from the unique set
Response_delete = pd.Series(Response_unique).sample(n=640403, random_state=42)


3090403


In [41]:
df_country_98 = df_country[~df_country['ResponseID'].isin(Response_delete)]

In [42]:
# checking shape of df_filtered

df_country_98.shape   # should be 4.9M

(4900000, 41)

In [44]:
df_country_98.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
0,2224kBG72574tbZD3,737909459_839962439872333.0,8.399624e+14,12,0,0,1,0,Old,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2228EQY3XnmTh3dq8,-1700464608_947225429117679.0,9.472254e+14,7,0,0,0,0,More,Utilitarian,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2229qWoicHtyXWHJd,596330438_8731233368451661.0,8.731233e+15,6,0,1,0,0,Young,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,222HpiEf2LtAwEg62,-1232628507_1597557389,1.597557e+09,5,0,0,1,0,Female,Gender,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,222bh6ceSDJvxhaak,329642844_496228042967179.0,4.962280e+14,4,0,1,0,0,Pets,Species,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [45]:
# check what the distribution of RepsonseID's is (kinda)
# check if the indices of a responseID here are the same after transforming the ResponseID column
# if the indices are the same means ResponseID column is succesfully transformed

indices = df_country_98[df_country_98["ResponseID"] == '2224kBG72574tbZD3'].index
print(indices)

Index([0, 884580], dtype='int64')


In [47]:
# Changing the responseID

# Define the starting point for the new ResponseID
starting_id = 146784

# Step 1: Get the unique ResponseIDs
unique_response_ids = df_country_98['ResponseID'].unique()

# Step 2: Create a mapping from old ResponseID to new 'res_' formatted ID
response_id_mapping = {old_id: f'res_{i:08d}' for i, old_id in enumerate(unique_response_ids, starting_id)}

# Step 3: Replace the original ResponseID with the new mapped IDs
df_country_98['ResponseID'] = df_country_98['ResponseID'].map(response_id_mapping)

C:\Users\esmku\AppData\Local\Temp\ipykernel_28492\1528720168.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country_98['ResponseID'] = df_country_98['ResponseID'].map(response_id_mapping)


In [48]:
# ResponseID starts with res_00146784
df_country_98.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
0,res_00146784,737909459_839962439872333.0,8.399624e+14,12,0,0,1,0,Old,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,res_00146785,-1700464608_947225429117679.0,9.472254e+14,7,0,0,0,0,More,Utilitarian,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,res_00146786,596330438_8731233368451661.0,8.731233e+15,6,0,1,0,0,Young,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,res_00146787,-1232628507_1597557389,1.597557e+09,5,0,0,1,0,Female,Gender,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,res_00146788,329642844_496228042967179.0,4.962280e+14,4,0,1,0,0,Pets,Species,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [49]:
indices = df_country_98[df_country_98["ResponseID"] == 'res_00146784'].index
print(indices)

# checking if the indices are the same after transforming the ResponseID column
# compared to the indices of the ResponseID before transforming the column (which was 2224kBG72574tbZD3) - they are!
# both are [2, 884580]

Index([0, 884580], dtype='int64')


In [50]:
df_country_98.tail()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
6999991,res_02596779,1615899623_514570710,5.145707e+08,5,1,0,1,0,Pets,Species,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
6999995,res_02596780,-1882580163_3293063696269629.0,3.293064e+15,2,1,0,1,0,Fat,Fitness,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6999996,res_02596781,-1863857309_1456185995,1.456186e+09,6,1,1,0,1,Young,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6999997,res_02596782,1946363904_4846540250658522.0,4.846540e+15,1,1,1,0,1,Female,Gender,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6999999,res_02596783,434100736_1451013350,1.451013e+09,1,1,1,0,0,More,Utilitarian,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
df_country_98.columns

Index(['ResponseID', 'ExtendedSessionID', 'UserID', 'ScenarioOrder',
       'Intervention', 'PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel',
       'ScenarioTypeStrict', 'ScenarioType', 'DefaultChoice',
       'NonDefaultChoice', 'DefaultChoiceIsOmission', 'NumberOfCharacters',
       'DiffNumberOFCharacters', 'Saved', 'Template', 'DescriptionShown',
       'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
       'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
       'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
       'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
       'Cat'],
      dtype='object')

In [52]:
# already deleting some columns to ensure it fits in memory
# deleting the columns that are not necessary for the modelling: ExtendedSessionID, DefaultChoice, NonDefaultChoice, DefaultChoiceIsOmission, Template

df_country_clean = df_country_98.drop(columns=['ExtendedSessionID', 'DefaultChoice', 'NonDefaultChoice', 'DefaultChoiceIsOmission', 'Template', 'ScenarioType', 'ScenarioOrder', 'DescriptionShown', 'LeftHand', 'UserCountry3'])

In [53]:
df_country_clean.shape # should be 31 columns

(4900000, 31)

In [4]:
# resetting the index
df_country_clean = df_country_clean.reset_index(drop=True)

In [3]:
# binarizing UserID - making all into 0
df_country_clean['UserID'] = 0

In [5]:
# saving this pre-preprocessed non-western dataset to a csv file

df_country_clean.to_csv('non_western_dataset.csv', index=False)